In [18]:
import query.datasets.prelude
reload(query.datasets.prelude)
from query.datasets.prelude import *
from query.datasets.tvnews.shot_detect import shot_detect, shot_stitch
from query.datasets.tvnews.face_detect import face_detect
from query.datasets.tvnews.face_embed import face_embed
from query.datasets.tvnews.pose_detect import pose_detect
from query.datasets.tvnews.identity_detect import identity_detect
from query.datasets.tvnews.animatedness import shot_frame_to_detect

In [2]:
with Timer('Detecting shots'):
    import query.datasets.tvnews.shot_detect
    reload(query.datasets.tvnews.shot_detect)
    from query.datasets.tvnews.shot_detect import shot_detect
    log.debug('Loading videos')
    all_videos = list(tqdm(Video.objects.all().order_by('id')))
    shot_indices, all_shots, all_blackframes = shot_detect(all_videos)
    shot_videos = gather(all_videos, shot_indices)
    log.debug('Computing face frames to detect')
    face_frame_per_shot = [[shot_frame_to_detect(shot) for shot in vid_shots]
                           for vid_shots in tqdm(all_shots)]

D 18-01-17 23:26:51 prelude.py:229] -- START: Detecting shots
D 18-01-17 23:26:51 <ipython-input-2-5028577b130d>:005] Loading videos


HBox(children=(IntProgress(value=0, max=62046), HTML(value=u'')))

D 18-01-17 23:26:52 prelude.py:229] -- START: Loading from cache: all_shots_blackframes


D 18-01-17 23:29:07 prelude.py:236] -- END: Loading from cache: all_shots_blackframes -- 00:02:14
D 18-01-17 23:29:07 <ipython-input-2-5028577b130d>:009] Computing face frames to detect


HBox(children=(IntProgress(value=0, max=60720), HTML(value=u'')))

D 18-01-17 23:29:39 prelude.py:236] -- END: Detecting shots -- 00:02:48


In [3]:
with Timer('Detecting sparse face'):
    import query.datasets.tvnews.face_detect
    reload(query.datasets.tvnews.face_detect)
    from query.datasets.tvnews.face_detect import face_detect
    
    all_faces, indices = face_detect(shot_videos, face_frame_per_shot)
    face_videos = gather(shot_videos, indices)
    face_shots = gather(all_shots, indices)
    face_frames = gather(face_frame_per_shot, indices)
    print(len(all_faces))

D 18-01-17 23:29:39 prelude.py:229] -- START: Detecting sparse face
D 18-01-17 23:29:39 prelude.py:229] -- START: Loading from cache: all_bboxes
D 18-01-17 23:30:52 prelude.py:236] -- END: Loading from cache: all_bboxes -- 00:01:04
D 18-01-17 23:30:52 prelude.py:236] -- END: Detecting sparse face -- 00:01:13


60607


In [ ]:
with Timer('Saving faces'):
    frames_to_save = [
        [{
            'video_id': video.id,
            'number': f
        } for f in frames]
        for (video, frames) in zip(face_videos, face_frames)
    ]
    Frame.objects.bulk_create_copy(flatten(frames_to_save))
    
    labeler, _ = Labeler.objects.get_or_create(name='mtcnn')
    for (frames, vid_faces) in zip(frames_to_save, all_faces):
        for (frame, faces) in zip(frames, vid_faces):
            for face in faces:
                face['frame_id'] = frame['id']
                face['labeler_id'] = labeler.id
    Face.objects.bulk_create_copy(flatten(all_faces))

In [ ]:
make_montage(face_videos[0], face_frames[0], '/app/montage.jpg', bboxes=all_faces[0])

In [4]:
def output_name(video, frames):
    return video.path + '_faces_' + str(hash(tuple(frames)))

face_tables = [output_name(video, frames) for video, frames in tqdm(zip(face_videos, face_frames))]

HBox(children=(IntProgress(value=0, max=60607), HTML(value=u'')))

In [ ]:
with Timer('Gender faces'):
    import query.datasets.tvnews.gender_detect
    reload(query.datasets.tvnews.gender_detect)
    from query.datasets.tvnews.gender_detect import gender_detect
    
    gender_detect(face_videos, face_frames, face_tables)

In [ ]:
with Timer("Embedding faces"):
    import query.datasets.tvnews.face_embed
    reload(query.datasets.tvnews.face_embed)
    from query.datasets.tvnews.face_embed import face_embed

    face_embed(face_videos, face_frames, face_tables)

In [5]:
def output_name(video, frames):
    return video.path + '_embeddings_' + str(hash(tuple(frames)))

with make_scanner_db() as db:
    db._load_db_metadata()
    
indices, embed_tables = unzip([
    (i, db.table(output_name(video, frames)))
    for i, (video, frames) in tqdm(enumerate(zip(face_videos, face_frames)))
    if db.has_table(output_name(video, frames)) and db.table(output_name(video, frames)).committed()
])
print(len(indices))

embed_videos, embed_frames, embed_faces, embed_shots = map(lambda l: gather(l, indices),
                                                          (face_videos, face_frames, all_faces, face_shots))


HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))


60607


In [ ]:
with Timer("Embedding faces"):
    def load_embs():
        log.debug('Loading embs')
        EMBEDDING_SIZE = 128
        def load(t):
            embs = list(t.column('embeddings').load())
            arrays = [np.frombuffer(emb, dtype=np.float32) if emb is not None else [] for _, emb in embs]
            return [np.split(a, len(a) / 128) if len(a) > 0 else [] for a in arrays]
        return par_for(load, embed_tables, workers=8)

    all_embs = pcache.get('all_embs', load_embs, method='pickle', force=True)


In [7]:
with Timer("Stitching shots"):    
    def load_stitches():
        log.debug('Computing stitches')
        import query.datasets.tvnews.shot_detect
        reload(query.datasets.tvnews.shot_detect)
        return query.datasets.tvnews.shot_detect.shot_stitch(embed_videos, embed_shots, embed_frames, embed_faces, all_embs)
    (stitched_shots, stitched_indices) = pcache.get('stitched_shots', load_stitches)

D 18-01-17 23:39:22 prelude.py:229] -- START: Stitching shots
D 18-01-17 23:39:22 prelude.py:229] -- START: Loading from cache: stitched_shots
D 18-01-17 23:41:21 prelude.py:236] -- END: Loading from cache: stitched_shots -- 00:01:54
D 18-01-17 23:41:21 prelude.py:236] -- END: Stitching shots -- 00:01:58


In [9]:
all_embs = embed_faces
stitched_videos = embed_videos
stitched_frames, stitched_faces, stitched_embs = map(
    lambda t: [gather(l, idx) for l, idx in zip(t, stitched_indices)],
    (embed_frames, embed_faces, all_embs))

In [10]:
show = Show.objects.get(name='The Rachel Maddow Show').id
indices = [i for i, video in enumerate(stitched_videos) if video.show_id == show]

with Timer('Detecting identities'):
    def load_identities():
        log.debug('Computing identities')
        import query.datasets.tvnews.identity_detect
        reload(query.datasets.tvnews.identity_detect)

        return query.datasets.tvnews.identity_detect.identity_detect(
           gather(stitched_videos, indices), "/app/rachel-maddow.jpg", gather(stitched_embs, indices))
    
    matching_indices = pcache.get('matching_indices', load_identities)
    
print(len(matching_indices), sum([len(l) for l in matching_indices]))

D 18-01-17 23:49:14 prelude.py:229] -- START: Detecting identities
D 18-01-17 23:49:14 prelude.py:229] -- START: Loading from cache: matching_indices
D 18-01-17 23:49:14 prelude.py:236] -- END: Loading from cache: matching_indices -- 00:00:00
D 18-01-17 23:49:14 prelude.py:236] -- END: Detecting identities -- 00:00:00


(1935, 89523)


In [23]:
import random
flat_indices = [(i, j, k) for i, idx in enumerate(matching_indices) for (j, k) in idx]
random_indices = random.sample(flat_indices, 16)
identity_videos = gather(stitched_videos, indices)
identity_frames = gather(stitched_frames, indices)
make_montage(
    [identity_videos[i] for i, _1, _2 in random_indices],
    [identity_frames[i][j] for i, j, _ in random_indices],
    'montage.jpg',
    num_cols=4,
    target_height=240)

In [ ]:
matching_indices_onelevel = [[j for j, k in l] for l in matching_indices]
matching_videos, matching_shots, matching_frames, matching_faces = \
    (gather(filter3_videos, indices),      
    [gather(shots, idxs) for (shots, idxs) in zip(gather(stitched_shots, indices), matching_indices_onelevel)],
    [gather(frames, idxs) for (frames, idxs) in zip(gather(filter3_frames, indices), matching_indices_onelevel)],
    [gather2(faces, idxs) for (faces, idxs) in zip(gather(filter3_faces, indices), matching_indices)])
    
#pose_blacklist = ['tvnews/videos/MSNBCW_20170708_010000_The_Rachel_Maddow_Show.mp4']    
#pose_blacklist = ['tvnews/videos/MSNBCW_20170506_040000_The_Rachel_Maddow_Show.mp4']
pose_blacklist = []
    
matching2_videos, matching2_shots, matching2_frames, matching2_faces = unzip([
    (video, shots, frames, faces)
    for video, shots, frames, faces in zip(matching_videos, matching_shots, matching_frames, matching_faces)
    if len(shots) > 0 and video.path not in pose_blacklist
])
    
TARGET_FPS = 10
pose_frames = [
    sum([list(range(s['min_frame'], s['max_frame'], int(round(video.fps / TARGET_FPS)))) for s in shots], [])
    for (video, shots) in zip(matching2_videos, matching2_shots)    
]

In [ ]:
import query.datasets.tvnews.pose_detect
reload(query.datasets.tvnews.pose_detect)
from query.datasets.tvnews.pose_detect import pose_detect

all_poses = pose_detect(matching2_videos, pose_frames, force=True)

In [ ]:
import query.datasets.tvnews.pose_detect
reload(query.datasets.tvnews.pose_detect)
from query.datasets.tvnews.pose_detect import pose_track


#pose_track(matching2_videos, matching2_shots, matching2_frames, matching2_faces, all_poses)

In [ ]:
def bbox_montage((i, (video, frames, faces, matches))):
    if len(matches) == 0:
        print(video.path)
        return
    make_montage(
        video,
        [frames[j] for j, _ in matches],
        '/tmp/montage{}.jpg'.format(i),
        bboxes=[[faces[j][k]] for j, k in matches],
        progress=False)

In [ ]:
import query.datasets.prelude
reload(query.datasets.prelude)
from query.datasets.prelude import *

# make_montage(filter3_videos[indices[i]], filter3_frames[indices[i]],
#              '/app/montage.jpg', filter3_faces[indices[i]], workers=96, progress=True)

def bbox_montage((i, (video, frames, faces, matches))):
    try:
        if len(matches) == 0:
            print(video.path)
            return
        bbox_map = defaultdict(list, {j: [faces[j][k]] for j, k in matches})
        make_montage(
            video,
            #[frames[j] for j, _ in matches],
            frames,
            '/tmp/montage{}.jpg'.format(i),
            bboxes=[bbox_map[i] for i in range(len(frames))],
            #bboxes=[[faces[j][k]] for j, k in matches],
            progress=False)
    except Exception:
        traceback.print_exc()
        print(video.path)
    
_ = par_for(bbox_montage,
        list(enumerate(zip(gather(filter3_videos, indices)[:100], gather(filter3_frames, indices), 
            gather(filter3_faces, indices), matching_indices))),
        process=True,
        workers=8)

In [ ]:
import requests
import cv2

par_for(make_montage,
        list(enumerate(zip(filter2_videos, [[s['min_frame'] for s in l] for l in stitched_shots][:100]))),
        process=True,
        workers=8)

In [ ]:
Shot.objects.bulk_create_copy({
    'min_frame': shot['min_frame'], 
    'max_frame': shot['max_frame'], 
    'labeler_id': shot['labeler'],
    'video_id': shot['video__id']
} for shot_list in tqdm(stitched_shots) for shot in shot_list])